In [ ]:
def run_simulation_for_different_M(num_nodes_to_poll_list):
    average_aoii_results = []

    for M in num_nodes_to_poll_list:
        # Reinitialize trackers and states
        state_node, state_sink, last_update_times, aoii_time_tracker, aoii_history = initialize_states(num_nodes)

        # Run the simulation
        for t in range(len(pivot_df)):
            # For each time step, every node updates its state using dEWMA filter with new measurements
            for mote in range(1, num_nodes + 1):
                column_name = f'mote{mote}'
                measured_value = pivot_df.loc[t, column_name]
                last_state_value, last_rate_of_change = state_node[column_name]
                state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)

            # Poll nodes based on Whittle index and AoII
            state_node, state_sink, actual_values, predicted_values, last_update_times, aoii_time_tracker, aoii_history = poll_nodes_whittle(
                pivot_df, t, state_node, state_sink, actual_values, predicted_values, M, penalty, last_update_times, aoii_time_tracker, aoii_history
            )

        # Calculate the average AoII across all nodes
        aoii_times = [np.mean(history) for history in aoii_history.values() if history]  # Exclude empty histories
        average_aoii_time = np.mean(aoii_times) if aoii_times else 0.0
        average_aoii_results.append(average_aoii_time)

    return average_aoii_results
